In [132]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import random

import matplotlib.pyplot as plt
import seaborn as sns

In [133]:
import yfinance as yf
import requests

In [134]:
df = pd.read_csv("https://query.data.world/s/pzd7uazpmpyortmg6fsuxiaoizllgu?dws=00000", encoding='ISO-8859-1')
display(df.head())

,Public Company Name (As Listed on Exchange),B Corp Impact Data ID,Ticker 1,Ticker 2,Ticker 3,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified
0,"Allbirds, Inc.",001C000001SGhnyIAD,NASDAQGS:BIRD,NaN,NaN,2021.0,NaN,2016.0,active,NaN
1,Amalgamated Financial,001C000001K16JSIAZ,NASDAQGM:AMAL,NaN,NaN,2018.0,NaN,2017.0,active,NaN
2,"AppHarvest, Inc.",0013b00001o41JFAAY,NASDAQGS:APPH,NaN,NaN,2020.0,NaN,2019.0,delisted,2023.0
3,Arezzo&Co,0013b00001qHFddAAG,BOVESPA:ARZZ3,NaN,NaN,2011.0,NaN,2022.0,active,NaN
4,Arowana International,001C000001ZzPK4IAN,ASX: AWN,NaN,NaN,2013.0,2021.0,2018.0,delisted,NaN


In [135]:
df.drop(['B Corp Impact Data ID', 'Ticker 2', 'Ticker 3'], axis=1, inplace = True)
print(len(df))
df.dropna(axis=0, subset=['Public Company Name (As Listed on Exchange)'], inplace=True)
print(len(df))

999
68


In [136]:
display(df[df['Public Listing Status'] == 'delisted'])

,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified
2,"AppHarvest, Inc.",NASDAQGS:APPH,2020.0,NaN,2019.0,delisted,2023.0
4,Arowana International,ASX: AWN,2013.0,2021.0,2018.0,delisted,NaN
14,Freshlocal Solutions,TSE: LOCL,2019.0,2021.0,2013.0,delisted,NaN


In [137]:
temp = df.copy()
temp = temp[temp['Public Listing Status'] == 'active']

temp['Ticker 1'].replace(": ", ":", regex=True, inplace=True)


print(len(temp))
temp['Exchange'] = temp['Ticker 1'].str.split(":").str[0]
temp['Ticker2'] = temp['Ticker 1'].str.split(":").str[1]
temp.dropna(subset=["Ticker2"], inplace=True) # Remove any NaN tickers

display(temp.head())

65


,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified,Exchange,Ticker2
0,"Allbirds, Inc.",NASDAQGS:BIRD,2021.0,NaN,2016.0,active,NaN,NASDAQGS,BIRD
1,Amalgamated Financial,NASDAQGM:AMAL,2018.0,NaN,2017.0,active,NaN,NASDAQGM,AMAL
3,Arezzo&Co,BOVESPA:ARZZ3,2011.0,NaN,2022.0,active,NaN,BOVESPA,ARZZ3
5,Australian Ethical Investment Ltd.,ASX:AEF,2002.0,NaN,2014.0,active,NaN,ASX,AEF
6,Benefit Systems S.A.,WSE:BFT,2013.0,NaN,2018.0,active,NaN,WSE,BFT


### Stocks are from Global Exchanges - build and maintain an exchange conversion dictionary to interface with Yahoo Finance

In [138]:
exchange_dict = {'ASX': '.XA', 
                 'WSE': '.WA',
                 'BOVESPA': '.SA',
                 'ETHEX': '',
                 'TSX': '.TO',
                 'ENXTPA': '.PA',
                 'BME': '.MC',
                 'ENXTBR': '.BR',
                 'KLSE': '.KL',
                 'LSE': '.AQ',
                 'TWSE': '.TW',
                 'NZSE': '.NZ',
                 'SGX': '.SI',
                 'BIT': '.MI',
                 'TASE': '.TA',
                 'XMIL': '.MI',
                 'NZE': '.NZ',
                 'BVMF': '.SA',
                 'HKG': '.HK',
                 'LON': '.L',
                 'MIL': '.MI',
                 'MOVI3': '.SA',
                 'TSE': '.T',
                 'BRK': '.L',
                 'SNSE': '.SN',
                 'MOVIDA B3': '.SA',
                 'NYSE': '',
                 'NASDAQGS': '',
                 'NASDAQGM': '',
                 'NASDAQCM': ''
                }

exchange_map = {exchanges: extention
                 for exchanges, extention in exchange_dict.items()
                 for exchange in exchanges}

temp['Intl Extention'] = temp['Exchange'].map(exchange_map)

In [139]:
temp['Ticker'] = temp['Ticker2'] + temp['Intl Extention'] 
mask = temp['Ticker'].isna()

# Check to see if we missed any:
temp[mask]

,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified,Exchange,Ticker2,Intl Extention,Ticker
19,Heidelberg District Community Enterprise Limited,NSX:EIC,2019.0,NaN,2019.0,active,NaN,NSX,EIC,NaN,NaN
36,SIDERPERU,BVL:SIDERC1,2006.0,NaN,2022.0,active,NaN,BVL,SIDERC1,NaN,NaN
41,Urb-it AB (publ),OM:URBIT,2017.0,NaN,2020.0,active,NaN,OM,URBIT,NaN,NaN
48,Weleda AG Group,OTC-X:BEKB:Weleda N-PS,NaN,NaN,2021.0,active,NaN,OTC-X,BEKB,NaN,NaN
56,Vinedos Emiliana S.A.,XSGO:EMILIANA,1996.0,NaN,2023.0,active,NaN,XSGO,EMILIANA,NaN,NaN


In [140]:
#Find datafeed issues
tickers = temp['Ticker'].to_list()

count = 0
for t in tickers:
    try:
        ticker = yf.Ticker(t)
        info = None
    except:
        count +=1
        print(f"Cannot get info of {t}, it probably does not exist")

print("Total nans =", count, "which is", round(count/temp.shape[0],2)*100, '% of the total')


Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Total nans = 5 which is 8.0 % of the total


In [141]:
# Remove Nans from stock universe:

b_corp_df = temp[-mask].copy()
display(b_corp_df.head())


,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified,Exchange,Ticker2,Intl Extention,Ticker
0,"Allbirds, Inc.",NASDAQGS:BIRD,2021.0,NaN,2016.0,active,NaN,NASDAQGS,BIRD,,BIRD
1,Amalgamated Financial,NASDAQGM:AMAL,2018.0,NaN,2017.0,active,NaN,NASDAQGM,AMAL,,AMAL
3,Arezzo&Co,BOVESPA:ARZZ3,2011.0,NaN,2022.0,active,NaN,BOVESPA,ARZZ3,.SA,ARZZ3.SA
5,Australian Ethical Investment Ltd.,ASX:AEF,2002.0,NaN,2014.0,active,NaN,ASX,AEF,.XA,AEF.XA
6,Benefit Systems S.A.,WSE:BFT,2013.0,NaN,2018.0,active,NaN,WSE,BFT,.WA,BFT.WA


In [142]:
isolate_us = False
if isolate_us:

    exchanges = set(temp['Exchange'])
    print(exchanges)
    us_ex = ['NYSE', 'NASDAQGS','NASDAQGM', 'NASDAQCM']
    us = temp[temp['Exchange'].isin(us_ex)]
    print("Isolating US Companies")
    display(us.head())


### List of Companies with B-Corp Subsidiaries
Unilever (NYSE: UN): (Pukka (UK), Mãe Terra (Brazil), Sir Kensington’s (US), Ben & Jerry’s (US), Olly Nutrition (US), Seventh Generation (US), Sundial (US), T2 Tea (Australia), Graze (UK))

B2W (B3: BTOW3): Courrieros (Brazil)

Grupo Soma (B3: SOMA3): Companhia Hering (Brazil)

Arezzo&Co (B3: ARZZ3): Reserva (Brazil)

Procter & Gamble (NYSE: PG): (New Chapter (US))

AB Inbev: (4 Pines Brewing Company)

Banco Estado: (Banco Estado Microempresas, Caja Vecina)

Coca-Cola: (Innocent Drinks)

Fairfax Financial: (The Redwoods Group)

Gap: (Athleta, Hill City)

Kikkoman: (Country Life)

Lactalis: (Stonyfield Farm)

Land O’ Lakes: (Vermonta Creamery)

Nestle: (Essential Living Foods, Garden of Life, Lily's Kitchen)

Oppenheimer Funds Inc.: (SNW Asset Management)

Rakuten: (OverDrive)

The Hain Celestial Group: (Ella’s Kitchen)

Vina Concha y Toro: (Fetzer Vineyards)


Hortfruti S.A.: Hortifruti Chile

Danone SA: (Danone North America, Alpro, Danone Waters of America, Danone Canada, -Aguas Danone Argentina, Bledina, Danone Aqua Indonesia, Danone Egypt, Danone Iberia, Danone Manifesto Ventures, Danone UK, Grameen Danone Foods, Happy Family, Les 2 Vaches, Danone Waters Germany, Danone Dairy Ireland, Danone Waters Spain, Danone Netherlands, Danone Dairy Belgium, Nutricia Bago, Danone ELN Greater China, Danone Japan, Volvic, Danone Brazil)

Caixabank (Imagin Ban)

Banco Sabadell (Sogeviso)

iSun (SunCommon)

In [143]:
add_subs = True
public_subs = ['UL', 'SOMA3.SA', 'ABI.BR', 'PG', 'FFH.TO', 'KO', 'GPS', 'KIK.F', 'NESN.SW',
               'OPY', '4755.T', 'HAIN', 'DANOY', 'CABK.MC', 'ISUN']

In [144]:
def get_stock_data (df):
    tickers_list = df['Ticker'].tolist()
    if add_subs:
        tickers_list = tickers_list + public_subs
    tickers_data= {}
    
    for ticker in tickers_list:
        ticker_object = yf.Ticker(ticker)
        
        #convert info() output from dictionary to dataframe
        temp = pd.DataFrame.from_dict(ticker_object.info, orient="index")
        temp.reset_index(inplace=True)
        temp.columns = ["Attribute", "Recent"]

        # add (ticker, dataframe) to main dictionary
        tickers_data[ticker] = temp

    return tickers_data
        
        

In [145]:
if isolate_us:
    load_stock_data = get_stock_data (us)
else:
    load_stock_data = get_stock_data (b_corp_df)
    
print("Number of Stock Records Loaded", len(load_stock_data))

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SZL.XA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SZL.XA&crumb=FUh37WBIK1d


Number of Stock Records Loaded 71


In [146]:
# Clean df:
stock_data = pd.concat(load_stock_data) #turn from dict to df
stock_data = stock_data.reset_index()

del stock_data["level_1"] # clean up unnecessary column
stock_data.columns = ["Ticker", "Attribute", "Recent"] # update column names


attr_to_keep = ['shortName', 'longName', 'industry', 'sector','previousClose', 'sharesOutstanding', 'impliedSharesOutstanding',
               'marketCap',  'currency', 'country', 'website', 'enterpriseValue',
                'trailingEps', 'forwardEps', 'enterprisetoEbitda', 'ebitda', 'grossMargins', 'ebitdaMargins', 'operatingMargins']


stock_data = stock_data[stock_data["Attribute"].isin(attr_to_keep)].reset_index()
del stock_data["index"] # clean up unnecessary column



display(stock_data.head(15))

,Ticker,Attribute,Recent
0,BIRD,country,United States
1,BIRD,website,https://www.allbirds.com
2,BIRD,industry,Apparel Retail
3,BIRD,sector,Consumer Cyclical
4,BIRD,previousClose,0.64
5,BIRD,marketCap,95674016
6,BIRD,currency,USD
7,BIRD,enterpriseValue,63238092
8,BIRD,sharesOutstanding,102641000
9,BIRD,impliedSharesOutstanding,155188992


In [147]:
print("Number of Stocks:", len(stock_data['Ticker'].unique()))
display(stock_data['Ticker'].unique())

Number of Stocks: 68


array(['BIRD', 'AMAL', 'ARZZ3.SA', 'AEF.XA', 'BFT.WA', 'BIO.XA',
       'CWEB.TO', 'CLSA3.SA', 'MLMFI.PA', 'COUR', 'FLOW.TO', 'FREY.PA',
       'GFI.NZ', 'GROV', 'HAN.MC', 'HLZ.MC', 'INCLU.BR', 'IBAB.BR',
       'KPG.XA', 'KCT.AQ', 'KMD.NZ', 'LMND', 'LFG.XA', 'MOVI3.SA',
       'NTCO3.SA', '2897.TW', '1H2.SI', 'ALQP.PA', 'RETI.MI', '6088.T',
       'SML.NZ', 'COCO', 'VNT.MI', 'CONCHATORO.SN', 'VITL', 'VVPR',
       'VHT.XA', 'WRBY', 'ZVIA', 'ABB.XA', 'TIME.MI', 'CVT.NZ', 'DOX.MI',
       'SOMA3.SA', '1112.HK', 'HMGS.TA', '1346.HK', '0973.HK', 'MME.XA',
       'MPR.XA', 'TENG.L', 'TPX.L', 'XSG.L', 'XHS.MI', 'UL', 'ABI.BR',
       'PG', 'FFH.TO', 'KO', 'GPS', 'KIK.F', 'NESN.SW', 'OPY', '4755.T',
       'HAIN', 'DANOY', 'CABK.MC', 'ISUN'], dtype=object)

In [148]:
stock_data_filtered = stock_data.pivot(columns = 'Attribute', index='Ticker', values='Recent')
stock_data_filtered.columns

# More info:
#stock_data_filtered = stock_data_filtered[['shortName', 'longName','country', 'currency', 'previousClose',
#                                           'sharesOutstanding', 'impliedSharesOutstanding','marketCap', 'enterpriseValue',
#                                           'ebitda', 'forwardEps',  'trailingEps', 'grossMargins', 'operatingMargins', 'ebitdaMargins',
#                                             'industry', 'sector', 'website']]
# Only essential info:
stock_data_filtered = stock_data_filtered[['shortName', 'longName','country', 'currency', 'previousClose',
                                           'impliedSharesOutstanding','marketCap','industry', 'sector', 'website']]

display(stock_data_filtered.head(10))

Attribute,shortName,longName,country,currency,previousClose,impliedSharesOutstanding,marketCap,industry,sector,website
Ticker,,,,,,,,,,
0973.HK,L'OCCITANE,L'Occitane International S.A.,Luxembourg,HKD,31.85,1472169984,45931704320,Household & Personal Products,Consumer Defensive,https://group.loccitane.com
1112.HK,H&H INTL HLDG,Health and Happiness (H&H) International Holdi...,Hong Kong,HKD,10.86,668150016,7191549952,Packaged Foods,Consumer Defensive,https://www.hh.global
1346.HK,LEVER STYLE,Lever Style Corporation,Hong Kong,HKD,0.94,645899008,607145024,Apparel Manufacturing,Consumer Cyclical,https://www.leverstyle.com
1H2.SI,$ Olive Tree,Olive Tree Estates Limited,Singapore,SGD,0.078,124756000,9035910,Real Estate - Diversified,Real Estate,https://www.olivetreeestates.com
2897.TW,O-BANK CO LTD,"O-Bank Co., Ltd.",Taiwan,TWD,9.91,2716470016,26893051904,Banks - Regional,Financial Services,https://www.o-bank.com
4755.T,RAKUTEN GROUP INC,"Rakuten Group, Inc.",Japan,JPY,882.8,2142140032,1883155202048,Internet Retail,Consumer Cyclical,https://global.rakuten.com/corp
6088.T,SIGMAXYZ HOLDINGS INC,SIGMAXYZ Holdings Inc.,Japan,JPY,1731.0,42703200,72436482048,Specialty Business Services,Industrials,https://www.sigmaxyz.com
ABB.XA,AUSSIEBAND FPO,Aussie Broadband Limited,Australia,AUD,3.59,NaN,NaN,NaN,NaN,https://www.aussiebroadband.com.au
ABI.BR,AB INBEV,Anheuser-Busch InBev SA/NV,Belgium,EUR,55.52,282105984,108635209728,Beverages - Brewers,Consumer Defensive,https://www.ab-inbev.com


### Incorporate exchange rates to translate price and market cap into USD

In [149]:
# Get avaialable currencies
currency_dict = {}

url = "https://currency-conversion-and-exchange-rates.p.rapidapi.com/symbols"

headers = {"X-RapidAPI-Key": "af4b092441msh3f4304a0bce0366p1680a3jsn93d4c3a0659f",
           "X-RapidAPI-Host": "currency-conversion-and-exchange-rates.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

api_data = response.json()

    
for symbol, curr in api_data['symbols'].items():
    currency_dict[symbol] = [curr, 0]

#currency_dict
    

In [150]:
## Access Exchange Rates:

url = "https://currency-conversion-and-exchange-rates.p.rapidapi.com/latest"

#querystring = {"from":"USD","to":"EUR,GBP"}
querystring = {'base':'USD'}

headers = {"X-RapidAPI-Key": "af4b092441msh3f4304a0bce0366p1680a3jsn93d4c3a0659f",
           "X-RapidAPI-Host": "currency-conversion-and-exchange-rates.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

exch_data = response.json()

for key in exch_data: 
    print(key,":", exch_data[key]) 

timestamp : 1711480024
base : USD
success : True
date : 2024-03-26
rates : {'ANG': 1.802704, 'SVC': 8.75213, 'CAD': 1.357915, 'XCD': 2.70255, 'MVR': 15.44999, 'HRK': 6.882375, 'AUD': 1.53043, 'MWK': 1739.999528, 'XAG': 0.04096, 'MAD': 10.1185, 'PHP': 56.273997, 'NAD': 18.980092, 'GNF': 8595.000148, 'KES': 131.000186, 'MZN': 63.501945, 'BTN': 83.312236, 'MGA': 4399.999744, 'AZN': 1.698901, 'XAU': 0.000459, 'RON': 4.587902, 'CHF': 0.903499, 'EGP': 47.820899, 'BSD': 1.000244, 'TWD': 31.917302, 'GGP': 0.790829, 'LVL': 0.60489, 'MMK': 2100.419181, 'WST': 2.76697, 'ILS': 3.66732, 'BHD': 0.376852, 'GBP': 0.79181, 'TZS': 2555.000047, 'SDG': 601.00008, 'LAK': 20949.999942, 'DJF': 177.720065, 'BYN': 3.272881, 'LBP': 89750.00008, 'RWF': 1288, 'PEN': 3.706039, 'EUR': 0.92309, 'ZMK': 9001.195368, 'RSD': 108.214056, 'INR': 83.32705, 'MUR': 46.318005, 'BWP': 13.701419, 'GEL': 2.713024, 'KMF': 454.499403, 'UZS': 12624.999635, 'RUB': 92.484996, 'CUC': 1, 'BGN': 1.805925, 'JOD': 0.7089, 'NGN': 1395.0195

In [151]:
## Combine date from exch_data with names / symbols in currency_dict:

for symbol, rate in exch_data['rates'].items():
    for key in currency_dict.keys():
        if symbol == key:
            currency_dict[key][1] = rate

In [152]:
currency_dict

{'ANG': ['Netherlands Antillean Guilder', 1.802704],
 'SVC': ['Salvadoran Colón', 8.75213],
 'CAD': ['Canadian Dollar', 1.357915],
 'XCD': ['East Caribbean Dollar', 2.70255],
 'MVR': ['Maldivian Rufiyaa', 15.44999],
 'HRK': ['Croatian Kuna', 6.882375],
 'AUD': ['Australian Dollar', 1.53043],
 'MWK': ['Malawian Kwacha', 1739.999528],
 'XAG': ['Silver (troy ounce)', 0.04096],
 'MAD': ['Moroccan Dirham', 10.1185],
 'PHP': ['Philippine Peso', 56.273997],
 'NAD': ['Namibian Dollar', 18.980092],
 'GNF': ['Guinean Franc', 8595.000148],
 'KES': ['Kenyan Shilling', 131.000186],
 'MZN': ['Mozambican Metical', 63.501945],
 'BTN': ['Bhutanese Ngultrum', 83.312236],
 'MGA': ['Malagasy Ariary', 4399.999744],
 'AZN': ['Azerbaijani Manat', 1.698901],
 'XAU': ['Gold (troy ounce)', 0.000459],
 'RON': ['Romanian Leu', 4.587902],
 'CHF': ['Swiss Franc', 0.903499],
 'EGP': ['Egyptian Pound', 47.820899],
 'BSD': ['Bahamian Dollar', 1.000244],
 'TWD': ['New Taiwan Dollar', 31.917302],
 'GGP': ['Guernsey Poun

In [153]:
## Add exhange rates and calculate USD equivalents:

#exchange_map = {exchanges: extention
#                 for exchanges, extention in exchange_dict.items()
#                 for exchange in exchanges}


currency_map = {symbols: rate
                 for symbols, name in currency_dict.items()
                 for rate in name}



stock_data_filtered['USDExchangeRate'] = stock_data_filtered['currency'].map(currency_map)



In [154]:
# Check to see that exchange rates mapped correctly:
stock_data_filtered.head(15)

Attribute,shortName,longName,country,currency,previousClose,impliedSharesOutstanding,marketCap,industry,sector,website,USDExchangeRate
Ticker,,,,,,,,,,,
0973.HK,L'OCCITANE,L'Occitane International S.A.,Luxembourg,HKD,31.85,1472169984,45931704320,Household & Personal Products,Consumer Defensive,https://group.loccitane.com,7.823350
1112.HK,H&H INTL HLDG,Health and Happiness (H&H) International Holdi...,Hong Kong,HKD,10.86,668150016,7191549952,Packaged Foods,Consumer Defensive,https://www.hh.global,7.823350
1346.HK,LEVER STYLE,Lever Style Corporation,Hong Kong,HKD,0.94,645899008,607145024,Apparel Manufacturing,Consumer Cyclical,https://www.leverstyle.com,7.823350
1H2.SI,$ Olive Tree,Olive Tree Estates Limited,Singapore,SGD,0.078,124756000,9035910,Real Estate - Diversified,Real Estate,https://www.olivetreeestates.com,1.345565
2897.TW,O-BANK CO LTD,"O-Bank Co., Ltd.",Taiwan,TWD,9.91,2716470016,26893051904,Banks - Regional,Financial Services,https://www.o-bank.com,31.917302
4755.T,RAKUTEN GROUP INC,"Rakuten Group, Inc.",Japan,JPY,882.8,2142140032,1883155202048,Internet Retail,Consumer Cyclical,https://global.rakuten.com/corp,151.546007
6088.T,SIGMAXYZ HOLDINGS INC,SIGMAXYZ Holdings Inc.,Japan,JPY,1731.0,42703200,72436482048,Specialty Business Services,Industrials,https://www.sigmaxyz.com,151.546007
ABB.XA,AUSSIEBAND FPO,Aussie Broadband Limited,Australia,AUD,3.59,NaN,NaN,NaN,NaN,https://www.aussiebroadband.com.au,1.530430
ABI.BR,AB INBEV,Anheuser-Busch InBev SA/NV,Belgium,EUR,55.52,282105984,108635209728,Beverages - Brewers,Consumer Defensive,https://www.ab-inbev.com,0.923090


In [155]:
# Calculate USD price and USD marketcap
stock_data_filtered['previousCloseUSD'] = stock_data_filtered['previousClose'] / stock_data_filtered['USDExchangeRate']
stock_data_filtered['marketCapUSD'] = stock_data_filtered['marketCap'] / stock_data_filtered['USDExchangeRate']
stock_data_filtered.head(15)

Attribute,shortName,longName,country,currency,previousClose,impliedSharesOutstanding,marketCap,industry,sector,website,USDExchangeRate,previousCloseUSD,marketCapUSD
Ticker,,,,,,,,,,,,,
0973.HK,L'OCCITANE,L'Occitane International S.A.,Luxembourg,HKD,31.85,1472169984,45931704320,Household & Personal Products,Consumer Defensive,https://group.loccitane.com,7.823350,4.071146,5871104363.22036
1112.HK,H&H INTL HLDG,Health and Happiness (H&H) International Holdi...,Hong Kong,HKD,10.86,668150016,7191549952,Packaged Foods,Consumer Defensive,https://www.hh.global,7.823350,1.388152,919241750.912333
1346.HK,LEVER STYLE,Lever Style Corporation,Hong Kong,HKD,0.94,645899008,607145024,Apparel Manufacturing,Consumer Cyclical,https://www.leverstyle.com,7.823350,0.120153,77606782.772086
1H2.SI,$ Olive Tree,Olive Tree Estates Limited,Singapore,SGD,0.078,124756000,9035910,Real Estate - Diversified,Real Estate,https://www.olivetreeestates.com,1.345565,0.057968,6715327.761944
2897.TW,O-BANK CO LTD,"O-Bank Co., Ltd.",Taiwan,TWD,9.91,2716470016,26893051904,Banks - Regional,Financial Services,https://www.o-bank.com,31.917302,0.31049,842585375.919305
4755.T,RAKUTEN GROUP INC,"Rakuten Group, Inc.",Japan,JPY,882.8,2142140032,1883155202048,Internet Retail,Consumer Cyclical,https://global.rakuten.com/corp,151.546007,5.825294,12426293766.011269
6088.T,SIGMAXYZ HOLDINGS INC,SIGMAXYZ Holdings Inc.,Japan,JPY,1731.0,42703200,72436482048,Specialty Business Services,Industrials,https://www.sigmaxyz.com,151.546007,11.422274,477983442.005173
ABB.XA,AUSSIEBAND FPO,Aussie Broadband Limited,Australia,AUD,3.59,NaN,NaN,NaN,NaN,https://www.aussiebroadband.com.au,1.530430,2.345746,NaN
ABI.BR,AB INBEV,Anheuser-Busch InBev SA/NV,Belgium,EUR,55.52,282105984,108635209728,Beverages - Brewers,Consumer Defensive,https://www.ab-inbev.com,0.923090,60.145815,117686476646.914169


## Filter Universe of Stocks by USD MarketCap

In [156]:
def filter_index(df, size=True, num=True, size_thresh = 1000000000, num_thresh = 20):
    temp = df.copy()
    if size == True:
        temp = temp[temp['marketCapUSD'] >= size_thresh]
    
    if num == True:
        temp.sort_values(by=['marketCapUSD'], ascending=False, inplace = True)
        temp = temp.iloc[0:num_thresh,:]
            
    return temp



In [158]:
stock_index_top20 = filter_index(stock_data_filtered, size = False, num_thresh=20)
stock_index_top30 = filter_index(stock_data_filtered, size = False, num_thresh=30)
stock_index_over1b = filter_index(stock_data_filtered, size = True, num=False, size_thresh=1000000000)

print(stock_index_top20.shape, stock_index_top30.shape, stock_index_over1b.shape )

(20, 13) (30, 13) (20, 13)


In [135]:
def get_price_history2 (df, start, end):
    tickers_list = df['Ticker'].tolist()
    tickers_data= {}
    first_object = yf.Ticker(tickers_list[0])
    prices = pd.DataFrame.from_dict(first_object.history(start=start, end=end, interval = "1d"))
    prices.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
    prices.columns = [tickers_list[0]]
    
    for ticker in tickers_list[1:]:
        ticker_object = yf.Ticker(ticker)
        try:
            temp = pd.DataFrame.from_dict(ticker_object.history(start=start, end=end, interval = "1d"))
            temp.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
            temp.columns = [ticker]
            prices = prices.merge(temp, how="outer", left_index=True, right_index=True)
        except:
            continue

    return prices

In [136]:
yf.Ticker("NESN.SW").history(start='2022-01-01', end='2022-01-25', interval='1d')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00+01:00,122.214293,123.662425,122.099972,123.109848,3412729,0.0,0.0
2022-01-04 00:00:00+01:00,123.662420,123.662420,122.061851,123.376602,2920374,0.0,0.0
2022-01-05 00:00:00+01:00,122.138077,122.157135,119.718187,120.042107,5579892,0.0,0.0
2022-01-06 00:00:00+01:00,118.632082,119.908724,118.346272,119.775345,3806803,0.0,0.0
2022-01-07 00:00:00+01:00,119.394256,119.641964,118.174782,118.574921,3579390,0.0,0.0
2022-01-10 00:00:00+01:00,118.708299,118.727357,115.831103,116.135971,4560274,0.0,0.0
2022-01-11 00:00:00+01:00,116.478944,117.488820,115.964477,116.402725,3707378,0.0,0.0
2022-01-12 00:00:00+01:00,116.707593,116.860031,115.469060,116.040695,5324981,0.0,0.0
2022-01-13 00:00:00+01:00,116.040702,116.345571,115.316637,115.659615,4004810,0.0,0.0


In [137]:
prices = get_price_history2(stocks_updated, start='2022-01-01', end='2023-12-31')

KCT.AQ: Period '1mo' is invalid, must be one of ['1d', '5d']


In [138]:
print(prices.shape)

top20=prices.iloc[:,0:20]
print(top20.shape)
top30=prices.iloc[:,0:30]
print(top30.shape)

(4535, 67)
(4535, 20)
(4535, 30)


In [159]:
def calc_index2(df, num_in_index = 30, divisor = 1000000000):
    cols = df.columns
    mkt_cap_cols = []
    for col in cols:
        if "marketCapUSD" in col:
            mkt_cap_cols.append(col)
    
    mkt_cap_df = df[mkt_cap_cols].copy()
    
    mkt_cap_df = mkt_cap_df.iloc[0:num_in_index,:]
    #display(mkt_cap_df)
    
    # Sort by beginning mkt cap
    baseline_order = mkt_cap_df.iloc[0,:].sort_values(ascending=False)
    #baseline_all = baseline_order.index.to_list()

    
    mkt_cap_df['Total_Index'] = mkt_cap_df.sum(axis=0)/divisor
    
    return mkt_cap_df

In [160]:
calc_index2(stocks_updated)

,marketCapUSD,Total_Index
0,3.789540e+11,NaN
1,2.759200e+11,NaN
2,2.616030e+11,NaN
3,1.255920e+11,NaN
4,1.191740e+11,NaN
5,1.176780e+11,NaN
6,4.152268e+10,NaN
7,3.796921e+10,NaN
8,2.637475e+10,NaN
9,1.257604e+10,NaN


In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize = (20,18))
ax0.plot(normed_data0, color = 'seagreen', linewidth=2, label = 'With Subs')
ax0.legend(loc=3)
ax1.plot(normed_data1, color = 'royalblue', linewidth=2,label='Cert B Corps')
ax1.legend(loc=3)
ax2.plot(normed_data0, color = 'seagreen', linewidth=2, label = 'With Subs')
ax2.plot(normed_data1, color = 'royalblue', linewidth=2, label = 'Cert B Corps')
ax2.plot(normed_spx, color='darkorange', linewidth=2, label = 'SPX')
ax2.legend(loc=3)
plt.show()

In [105]:
## This is the old code.....

def get_price_history (df, start, end, add_subs = False):
    tickers_list = df['Ticker'].tolist()
    if add_subs:
        tickers_list = tickers_list + public_subs

    tickers_data= {}
    first_object = yf.Ticker(tickers_list[0])
    prices = pd.DataFrame.from_dict(first_object.history(start=start, end=end, interval = "1d"))
    prices.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
    prices.columns = [tickers_list[0]]
    
    for ticker in tickers_list[1:]:
        ticker_object = yf.Ticker(ticker)
        temp = pd.DataFrame.from_dict(ticker_object.history(start=start, end=end, interval = "1d"))
        temp.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
        temp.columns = [ticker]
        prices = prices.merge(temp, how="outer", left_index=True, right_index=True)

    return prices

In [104]:
#px_df = get_price_history(filtered, start="2022-01-01", end = "2024-02-23")
spx_object = yf.Ticker("^GSPC")
spx = pd.DataFrame.from_dict(spx_object.history(start="2022-01-01", end="2024-02-23", interval = "1d"))
spx

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00-05:00,4778.140137,4796.640137,4758.169922,4796.560059,3831020000,0.0,0.0
2022-01-04 00:00:00-05:00,4804.509766,4818.620117,4774.270020,4793.540039,4683170000,0.0,0.0
2022-01-05 00:00:00-05:00,4787.990234,4797.700195,4699.439941,4700.580078,4887960000,0.0,0.0
2022-01-06 00:00:00-05:00,4693.390137,4725.009766,4671.259766,4696.049805,4295280000,0.0,0.0
2022-01-07 00:00:00-05:00,4697.660156,4707.950195,4662.740234,4677.029785,4181510000,0.0,0.0
...,...,...,...,...,...,...,...
2024-02-15 00:00:00-05:00,5003.140137,5032.720215,4999.439941,5029.729980,4137970000,0.0,0.0
2024-02-16 00:00:00-05:00,5031.129883,5038.700195,4999.520020,5005.569824,3833270000,0.0,0.0
2024-02-20 00:00:00-05:00,4989.319824,4993.709961,4955.020020,4975.509766,4034880000,0.0,0.0


In [ ]:
def add_shares_out (df, add_subs = False):
    df['BIRD_SharesOut'] = 152850000
    df['BIRD_MktCap'] = df['BIRD_SharesOut'] * df['BIRD']

    df['AMAL_SharesOut'] = 30430000
    df['AMAL_MktCap'] = df['AMAL_SharesOut'] * df['AMAL']

    df['COUR_SharesOut'] = 156950000
    df['COUR_MktCap'] = df['COUR_SharesOut'] * df['COUR']

    df['GROV_SharesOut'] = 37450000
    df['GROV_MktCap'] = df['GROV_SharesOut'] * df['GROV']

    df['LMND_SharesOut'] = 69920000
    df['LMND_MktCap'] = df['LMND_SharesOut'] * df['LMND']

    df['COCO_SharesOut'] = 56800000
    df['COCO_MktCap'] = df['COCO_SharesOut'] * df['COCO']

    df['VITL_SharesOut'] = 41600000
    df['VITL_MktCap'] = df['VITL_SharesOut'] * df['VITL']

    df['VVPR_SharesOut'] = 3270000
    df['VVPR_MktCap'] = df['VVPR_SharesOut'] * df['VVPR']

    df['WRBY_SharesOut'] = 117640000
    df['WRBY_MktCap'] = df['WRBY_SharesOut'] * df['WRBY']

    df['ZVIA_SharesOut'] = 50410000
    df['ZVIA_MktCap'] = df['ZVIA_SharesOut'] * df['ZVIA']


    if add_subs:
        df['UL_SharesOut'] = 2499020000
        df['UL_MktCap'] = df['UL_SharesOut'] * df['UL']

        df['PG_SharesOut'] = 2353020000
        df['PG_MktCap'] = df['PG_SharesOut'] * df['PG']

        df['BUD_SharesOut'] = 1986770000
        df['BUD_MktCap'] = df['BUD_SharesOut'] * df['BUD']

        df['KO_SharesOut'] = 4312460000
        df['KO_MktCap'] = df['KO_SharesOut'] * df['KO']


        df['GPS_SharesOut'] = 370830000 
        df['GPS_MktCap'] = df['GPS_SharesOut'] * df['GPS']


        df['OPY_SharesOut'] = 10290000
        df['OPY_MktCap'] = df['OPY_SharesOut'] * df['OPY']


        df['HAIN_SharesOut'] = 89830000
        df['HAIN_MktCap'] = df['HAIN_SharesOut'] * df['HAIN']


        df['DANOY_SharesOut'] = 2499020000
        df['DANOY_MktCap'] = df['DANOY_SharesOut'] * df['DANOY']
        
    col_order = df.columns.sort_values().tolist()
    df = df[col_order]
    
    return df


In [ ]:
#col_order = px_df.columns.sort_values().tolist()
#px_df = px_df[col_order]


In [ ]:
#px_df.to_csv('Public_B_Corps.csv')

In [ ]:
## This is the old code


def calc_index(df, divisor = 1000000000):
    cols = df.columns
    mkt_cap_cols = []
    for col in cols:
        if "MktCap" in col:
            mkt_cap_cols.append(col)
    
    mkt_cap_df = df[mkt_cap_cols].copy()
    
    # Sort by beginning mkt cap
    baseline_order = mkt_cap_df.iloc[0,:].sort_values(ascending=False)
    baseline_all = baseline_order.index.to_list()
    
    mkt_cap_df['Total_Index'] = mkt_cap_df[baseline_all].sum(axis=1)/divisor
    
    return mkt_cap_df

In [ ]:
def separate_subs (df, show_chart = True):

    px0 = get_price_history(df, start="2022-01-01", end = "2024-02-23", add_subs = True)
    px0 = add_shares_out(px0, add_subs = True)
    #px0.to_csv('24_03_19_BCorp_Index.csv')
    mkt_cap0 = calc_index(px0, divisor = 100000000)
    index0 = mkt_cap0[['Total_Index']]
    normed_data0 = index0.copy()
    normed_data0 = normed_data0.div(normed_data0.iloc[0]).fillna(0)


    px1 = get_price_history(df, start="2022-01-01", end = "2024-02-23", add_subs = False)
    px1 = add_shares_out(px1, add_subs = False)
    mkt_cap1 = calc_index(px1, divisor = 10000000)
    index1 = mkt_cap1[['Total_Index']]
    normed_data1 = index1.copy()
    normed_data1 = normed_data1.div(normed_data1.iloc[0]).fillna(0)
    
    # Add SPX:
    normed_spx = spx['Close'].copy()
    normed_spx = normed_spx.div(normed_spx.iloc[0]).fillna(0)
    
    if show_chart:
        fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize = (20,18))
        ax0.plot(normed_data0, color = 'seagreen', linewidth=2, label = 'With Subs')
        ax0.legend(loc=3)
        ax1.plot(normed_data1, color = 'royalblue', linewidth=2,label='Cert B Corps')
        ax1.legend(loc=3)
        ax2.plot(normed_data0, color = 'seagreen', linewidth=2, label = 'With Subs')
        ax2.plot(normed_data1, color = 'royalblue', linewidth=2, label = 'Cert B Corps')
        ax2.plot(normed_spx, color='darkorange', linewidth=2, label = 'SPX')
        ax2.legend(loc=3)
        plt.show()
        
    return px0, px1

In [ ]:
all_px, bcorp_px = separate_subs(us, show_chart = True)

In [ ]:
all_px.to_csv("B_Corp_Stocks.csv")

## Construct a Subset - look at reweighting?

In [ ]:
def determine_reweighting (df, n=8):
    cols = df.columns
    mkt_cap_cols = []
    for col in cols:
        if "MktCap" in col:
            mkt_cap_cols.append(col)
    mkt_cap_df = df[mkt_cap_cols].copy()
    
    # Sort by beginning mkt cap
    baseline_order = mkt_cap_df.iloc[0,:].sort_values(ascending=False)
    baseline_all = baseline_order.index.to_list()
    baseline_n = baseline_order.index[0:n].to_list()
    
    #Add Quarterly Flag
    mkt_cap_df['Quarter'] = mkt_cap_df.index.quarter
    
    mkt_cap_df['Reweight'] = mkt_cap_df['Quarter'] - mkt_cap_df['Quarter'].shift(periods=1)
    
    quarterly = mkt_cap_df[mkt_cap_df['Reweight'].isin([1,-3])]
    
    for i in range(quarterly.shape[0]):
        order = quarterly.iloc[i,:].sort_values(ascending=False)
        order_all = order.index.to_list()
        order_n = order.index[0:n].to_list()
        for j in order_n:
            if j not in baseline_n:
                print("Need to Reweight!", i, j)
                print("Baseline n", baseline_n)
                print("New n", order_n)
        baseline_n = order_n
    
    return quarterly
    #mkt_cap_df['Total_Index'] = mkt_cap_df[baseline_all].sum(axis=1)/10000000
    #mkt_cap_df['Total_Index_8'] = mkt_cap_df[baseline_8].sum(axis=1)/10000000

               

In [ ]:
quarterly = determine_reweighting(us, n = 12)

### Visualization

#### Seaborn color maps:
‘Accent’, ‘Accent_r’, ‘Blues’, ‘Blues_r’, ‘BrBG’, ‘BrBG_r’, ‘BuGn’, ‘BuGn_r’, ‘BuPu’, ‘BuPu_r’, ‘CMRmap’, ‘CMRmap_r’, ‘Dark2’, ‘Dark2_r’, ‘GnBu’, ‘GnBu_r’, ‘Greens’, ‘Greens_r’, ‘Greys’, ‘Greys_r’, ‘OrRd’, ‘OrRd_r’, ‘Oranges’, ‘Oranges_r’, ‘PRGn’, ‘PRGn_r’, ‘Paired’, ‘Paired_r’, ‘Pastel1’, ‘Pastel1_r’, ‘Pastel2’, ‘Pastel2_r’, ‘PiYG’, ‘PiYG_r’, ‘PuBu’, ‘PuBuGn’, ‘PuBuGn_r’, ‘PuBu_r’, ‘PuOr’, ‘PuOr_r’, ‘PuRd’, ‘PuRd_r’, ‘Purples’, ‘Purples_r’, ‘RdBu’, ‘RdBu_r’, ‘RdGy’, ‘RdGy_r’, ‘RdPu’, ‘RdPu_r’, ‘RdYlBu’, ‘RdYlBu_r’, ‘RdYlGn’, ‘RdYlGn_r’, ‘Reds’, ‘Reds_r’, ‘Set1’, ‘Set1_r’, ‘Set2’, ‘Set2_r’, ‘Set3’, ‘Set3_r’, ‘Spectral’, ‘Spectral_r’, ‘Wistia’, ‘Wistia_r’, ‘YlGn’, ‘YlGnBu’, ‘YlGnBu_r’, ‘YlGn_r’, ‘YlOrBr’, ‘YlOrBr_r’, ‘YlOrRd’, ‘YlOrRd_r’, ‘afmhot’, ‘afmhot_r’, ‘autumn’, ‘autumn_r’, ‘binary’, ‘binary_r’, ‘bone’, ‘bone_r’, ‘brg’, ‘brg_r’, ‘bwr’, ‘bwr_r’, ‘cividis’, ‘cividis_r’, ‘cool’, ‘cool_r’, ‘coolwarm’, ‘coolwarm_r’, ‘copper’, ‘copper_r’, ‘crest’, ‘crest_r’, ‘cubehelix’, ‘cubehelix_r’, ‘flag’, ‘flag_r’, ‘flare’, ‘flare_r’, ‘gist_earth’, ‘gist_earth_r’, ‘gist_gray’, ‘gist_gray_r’, ‘gist_heat’, ‘gist_heat_r’, ‘gist_ncar’, ‘gist_ncar_r’, ‘gist_rainbow’, ‘gist_rainbow_r’, ‘gist_stern’, ‘gist_stern_r’, ‘gist_yarg’, ‘gist_yarg_r’, ‘gnuplot’, ‘gnuplot2’, ‘gnuplot2_r’, ‘gnuplot_r’, ‘gray’, ‘gray_r’, ‘hot’, ‘hot_r’, ‘hsv’, ‘hsv_r’, ‘icefire’, ‘icefire_r’, ‘inferno’, ‘inferno_r’, ‘jet’, ‘jet_r’, ‘magma’, ‘magma_r’, ‘mako’, ‘mako_r’, ‘nipy_spectral’, ‘nipy_spectral_r’, ‘ocean’, ‘ocean_r’, ‘pink’, ‘pink_r’, ‘plasma’, ‘plasma_r’, ‘prism’, ‘prism_r’, ‘rainbow’, ‘rainbow_r’, ‘rocket’, ‘rocket_r’, ‘seismic’, ‘seismic_r’, ‘spring’, ‘spring_r’, ‘summer’, ‘summer_r’, ‘tab10’, ‘tab10_r’, ‘tab20’, ‘tab20_r’, ‘tab20b’, ‘tab20b_r’, ‘tab20c’, ‘tab20c_r’, ‘terrain’, ‘terrain_r’, ‘turbo’, ‘turbo_r’, ‘twilight’, ‘twilight_r’, ‘twilight_shifted’, ‘twilight_shifted_r’, ‘viridis’, ‘viridis_r’, ‘vlag’, ‘vlag_r’, ‘winter’, ‘winter_r’

Attributes = ['address1',
 'city',
 'state',
 'zip',
 'country',
 'phone',
 'website',
 'industry',
 'industryKey',
 'industryDisp',
 'sector',
 'sectorKey',
 'sectorDisp',
 'longBusinessSummary',
 'fullTimeEmployees',
 'companyOfficers',
 'auditRisk',
 'boardRisk',
 'compensationRisk',
 'shareHolderRightsRisk',
 'overallRisk',
 'governanceEpochDate',
 'compensationAsOfEpochDate',
 'maxAge',
 'priceHint',
 'previousClose',
 'open',
 'dayLow',
 'dayHigh',
 'regularMarketPreviousClose',
 'regularMarketOpen',
 'regularMarketDayLow',
 'regularMarketDayHigh',
 'beta',
 'forwardPE',
 'volume',
 'regularMarketVolume',
 'averageVolume',
 'averageVolume10days',
 'averageDailyVolume10Day',
 'bid',
 'ask',
 'bidSize',
 'askSize',
 'marketCap',
 'fiftyTwoWeekLow',
 'fiftyTwoWeekHigh',
 'priceToSalesTrailing12Months',
 'fiftyDayAverage',
 'twoHundredDayAverage',
 'currency',
 'enterpriseValue',
 'profitMargins',
 'floatShares',
 'sharesOutstanding',
 'sharesShort',
 'sharesShortPriorMonth',
 'sharesShortPreviousMonthDate',
 'dateShortInterest',
 'sharesPercentSharesOut',
 'heldPercentInsiders',
 'heldPercentInstitutions',
 'shortRatio',
 'shortPercentOfFloat',
 'impliedSharesOutstanding',
 'bookValue',
 'priceToBook',
 'lastFiscalYearEnd',
 'nextFiscalYearEnd',
 'mostRecentQuarter',
 'netIncomeToCommon',
 'trailingEps',
 'forwardEps',
 'pegRatio',
 'enterpriseToRevenue',
 'enterpriseToEbitda',
 '52WeekChange',
 'SandP52WeekChange',
 'exchange',
 'quoteType',
 'symbol',
 'underlyingSymbol',
 'shortName',
 'longName',
 'firstTradeDateEpochUtc',
 'timeZoneFullName',
 'timeZoneShortName',
 'uuid',
 'messageBoardId',
 'gmtOffSetMilliseconds',
 'currentPrice',
 'targetHighPrice',
 'targetLowPrice',
 'targetMeanPrice',
 'targetMedianPrice',
 'recommendationMean',
 'recommendationKey',
 'numberOfAnalystOpinions',
 'totalCash',
 'totalCashPerShare',
 'ebitda',
 'totalDebt',
 'quickRatio',
 'currentRatio',
 'totalRevenue',
 'debtToEquity',
 'revenuePerShare',
 'returnOnAssets',
 'returnOnEquity',
 'freeCashflow',
 'operatingCashflow',
 'revenueGrowth',
 'grossMargins',
 'ebitdaMargins',
 'operatingMargins',
 'financialCurrency',
 'trailingPegRatio']